In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [2]:
class cifar10vgg:
    def __init__(self,train=False):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [3]:
if __name__ == '__main__':


    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = cifar10vgg()

    predicted_x = model.predict(x_test)
    residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)


the validation 0/1 loss is:  0.0641


In [12]:
def write_cnn_to_file(X, name):
    dims1 = X.shape[0]
    dims2 = X.shape[1]
    dims3 = X.shape[2]
    dims4 = X.shape[3]
    
    f = open("./weights/"+name+".txt",'w')
    
    for c_o in range(0,dims4):
        for c_i in range(0,dims3):
            for h in range(0,dims2):
                for w in range(0,dims1):
                    f.write("{0:.15}\n".format(X[w,h,c_i,c_o]))  
    f.close()
    
    
    
def write_fcn_to_file(X, name):
    dims1 = X.shape[0]
    dims2 = X.shape[1]
    
    
    f = open("./weights/"+name+".txt",'w')
    
    for c in range(0,dims2):
        for i in range(0,dims1):
                f.write("{0:.15}\n".format(X[i,c]))  
    f.close()
    
    
    
def write_image_to_file(X, name):
    dims1 = X.shape[0]
    dims2 = X.shape[1]
    dims3 = X.shape[2]
    
    f = open("./data/"+name+".txt",'w')
    
    for c in range(0,dims3):
        for h in range(0,dims2):
            for w in range(0,dims1):
                f.write("{0:.15}\n".format(X[w,h,c]))  
    f.close()
    

In [4]:
import sys
cnn = model.model

#writing CNN weights
#Writing weights and biases for CNN
#for i in range(1,14):
  #  write_cnn_to_file(cnn.get_layer('conv2d_'+str(i)).get_weights()[0],'w'+str(i))

#for i in range(1,14):
    #tmp = cnn.get_layer('conv2d_'+str(i)).get_weights()[1]
   # tmp = tmp.reshape(tmp.shape[0],1)
    #print(tmp.shape)
   # np.savetxt('./weights/b'+str(i)+'.txt', tmp, delimiter=',')


#Writing weights and biases for FCN
#for i in range(1,3):
   # write_fcn_to_file(cnn.get_layer('dense_'+str(i)).get_weights()[0],'w'+str(i+13))
    #tmp = cnn.get_layer('dense_'+str(i)).get_weights()[1]
    #tmp = tmp.reshape(tmp.shape[0],1)
   # np.savetxt('./weights/b'+str(i+13)+'.txt', tmp, delimiter=',')

#Writing batch norm data
#for i in range(1,15):
    #[gamma , beta , moving_mean, moving_variance]
    #np.savetxt('./weights/gamma'+str(i)+'.txt', cnn.get_layer('batch_normalization_'+str(i)).get_weights()[0], delimiter=',')
    #np.savetxt('./weights/beta'+str(i)+'.txt', cnn.get_layer('batch_normalization_'+str(i)).get_weights()[1], delimiter=',')
    #np.savetxt('./weights/mean'+str(i)+'.txt', cnn.get_layer('batch_normalization_'+str(i)).get_weights()[2], delimiter=',')
    #np.savetxt('./weights/variance'+str(i)+'.txt', cnn.get_layer('batch_normalization_'+str(i)).get_weights()[3], delimiter=',')


#Helpful functions
print(cnn.summary())
#for i in range(0,14):
#    print(cnn.get_layer('batch_normalization_'+str(i+1)).get_weights()[0].shape)
#print(len(cnn.get_layer('batch_normalization_1').get_weights()))
#print(cnn.get_layer('batch_normalization_3').get_weights()[0])
#print(cnn.get_layer('dense_2').get_weights()[0].shape[1])
#print(cnn.get_layer('dense_2').get_weights()[1].shape)
#Batch Norm Data
#[gamma , beta , moving_mean, moving_variance]
#64     ,64    ,64          ,64  -> 64 channels!
#w= model.model.get_layer('batch_normalization_1').get_weights()

#print(x_test[0:200,:].shape)
#x_test_normalized = model.normalize_production(x_test[0:200,:])
#for i in range(0,200):
    #write_image_to_file(x_test_normalized[i,:], 'image'+str(i+1))

#print(np.argmax(model.predict(x_test[0:10,:]),1))


#print(len(cnn.get_layer('conv2d_1').get_weights()))








Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)       

In [5]:

weight_decay = 0.0005

tm = Sequential()#test tm

tm.add(Conv2D(64, (3, 3), padding='same',input_shape=[32,32,3],kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.3))
tm.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(MaxPooling2D(pool_size=(2, 2)))


tm.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.4))
tm.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(MaxPooling2D(pool_size=(2, 2)))

tm.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.4))
tm.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.4))
tm.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(MaxPooling2D(pool_size=(2, 2)))


tm.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.4))
tm.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.4))
tm.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(MaxPooling2D(pool_size=(2, 2)))


tm.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.4))
tm.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.4))
tm.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(MaxPooling2D(pool_size=(2, 2)))
tm.add(Dropout(0.5))

"""tm.add(Flatten())
tm.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
tm.add(Activation('relu'))
tm.add(BatchNormalization())
tm.add(Dropout(0.5))
tm.add(Dense(10))
tm.add(Activation('softmax'))"""


"tm.add(Flatten())\ntm.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))\ntm.add(Activation('relu'))\ntm.add(BatchNormalization())\ntm.add(Dropout(0.5))\ntm.add(Dense(10))\ntm.add(Activation('softmax'))"

In [8]:
tm.get_layer('conv2d_14').set_weights(cnn.get_layer('conv2d_1').get_weights())
tm.get_layer('conv2d_15').set_weights(cnn.get_layer('conv2d_2').get_weights())
tm.get_layer('conv2d_16').set_weights(cnn.get_layer('conv2d_3').get_weights())
tm.get_layer('conv2d_17').set_weights(cnn.get_layer('conv2d_4').get_weights())
tm.get_layer('conv2d_18').set_weights(cnn.get_layer('conv2d_5').get_weights())
tm.get_layer('conv2d_19').set_weights(cnn.get_layer('conv2d_6').get_weights())
tm.get_layer('conv2d_20').set_weights(cnn.get_layer('conv2d_7').get_weights())
tm.get_layer('conv2d_21').set_weights(cnn.get_layer('conv2d_8').get_weights())
tm.get_layer('conv2d_22').set_weights(cnn.get_layer('conv2d_9').get_weights())
tm.get_layer('conv2d_23').set_weights(cnn.get_layer('conv2d_10').get_weights())
tm.get_layer('conv2d_24').set_weights(cnn.get_layer('conv2d_11').get_weights())
tm.get_layer('conv2d_25').set_weights(cnn.get_layer('conv2d_12').get_weights())
tm.get_layer('conv2d_26').set_weights(cnn.get_layer('conv2d_13').get_weights())

tm.get_layer('batch_normalization_15').set_weights(cnn.get_layer('batch_normalization_1').get_weights())
tm.get_layer('batch_normalization_16').set_weights(cnn.get_layer('batch_normalization_2').get_weights())
tm.get_layer('batch_normalization_17').set_weights(cnn.get_layer('batch_normalization_3').get_weights())
tm.get_layer('batch_normalization_18').set_weights(cnn.get_layer('batch_normalization_4').get_weights())
tm.get_layer('batch_normalization_19').set_weights(cnn.get_layer('batch_normalization_5').get_weights())
tm.get_layer('batch_normalization_20').set_weights(cnn.get_layer('batch_normalization_6').get_weights())
tm.get_layer('batch_normalization_21').set_weights(cnn.get_layer('batch_normalization_7').get_weights())
tm.get_layer('batch_normalization_22').set_weights(cnn.get_layer('batch_normalization_8').get_weights())
tm.get_layer('batch_normalization_23').set_weights(cnn.get_layer('batch_normalization_9').get_weights())
tm.get_layer('batch_normalization_24').set_weights(cnn.get_layer('batch_normalization_10').get_weights())
tm.get_layer('batch_normalization_25').set_weights(cnn.get_layer('batch_normalization_11').get_weights())
tm.get_layer('batch_normalization_26').set_weights(cnn.get_layer('batch_normalization_12').get_weights())
tm.get_layer('batch_normalization_27').set_weights(cnn.get_layer('batch_normalization_13').get_weights())
#tm.get_layer('batch_normalization_28').set_weights(cnn.get_layer('batch_normalization_14').get_weights())

"""tm.get_layer('dense_3').set_weights(cnn.get_layer('dense_1').get_weights())
tm.get_layer('dense_4').set_weights(cnn.get_layer('dense_2').get_weights())"""

"tm.get_layer('dense_3').set_weights(cnn.get_layer('dense_1').get_weights())\ntm.get_layer('dense_4').set_weights(cnn.get_layer('dense_2').get_weights())"

In [12]:
print(tm.get_layer('batch_normalization_15').get_weights()[0][2])
print(tm.get_layer('batch_normalization_15').get_weights()[1][2])
print(tm.get_layer('batch_normalization_15').get_weights()[2][2])
print(tm.get_layer('batch_normalization_15').get_weights()[3][2])

-1.1869334
-0.039582044
0.83803517
0.02177414


In [13]:

mean = 120.707
std = 64.15
input = (x_test[0:200,:]-mean)/(std+1e-7)

for i in range(0,200):
    write_image_to_file(input[i,:], 'image'+str(i+1))
    
input = (x_test[0:1,:]-mean)/(std+1e-7)

#print(tm.summary())
np.set_printoptions(precision=8)
np.set_printoptions(threshold=sys.maxsize)
#print(m.get_layer('conv2d_20').get_weights()[0][0,2,0,0])
print(tm.predict(input,1)[0,:,0,0])
print(tm.predict(input,1)[0,:,0,1])
print(tm.predict(input,1)[0,:,0,2])
print(tm.predict(input,1)[0,:,0,3])

[-0.08577031]
[1.6536744]
[-0.55397797]
[3.4683394]
